In [0]:
import pandas as pd
import os.path
import cPickle as cpickle
import numpy as np
import keras.utils
import time
from keras.callbacks import TensorBoard, CSVLogger

Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [0]:
data_set = pd.read_csv('train.csv')

In [0]:
data_set.head()

,ID,label,statement,subject,speaker,speaker_job_title,state_info,part_affiliation,true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30.1,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w..."
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1.0,0,0,0,a news conference,She cited layoff notices received by the state...
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114.0,51,37,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he..."
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1.0,3,1,1,a radio show,"But spending still goes up. In addition, many ..."
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7.0,2,2,7,a web video,Our rating A Democratic Party web video making...


In [0]:
data_set.head(5)

,label,statement,subject,speaker,job,state,party,venue
0,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,Denver
3,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN


In [0]:
label_dict = {'pants-fire':0,'false':1,'barely-true':2,'half-true':3,'mostly-true':4,'true':5}
label_reverse_arr = ['pants-fire','false','barely-true','half-true','mostly-true','true']

In [0]:
speakers = ['barack-obama', 'donald-trump', 'hillary-clinton', 'mitt-romney', 
            'scott-walker', 'john-mccain', 'rick-perry', 'chain-email', 
            'marco-rubio', 'rick-scott', 'ted-cruz', 'bernie-s', 'chris-christie', 
            'facebook-posts', 'charlie-crist', 'newt-gingrich', 'jeb-bush', 
            'joe-biden', 'blog-posting','paul-ryan']
speaker_dict = {}
for cnt,speaker in enumerate(speakers):
    speaker_dict[speaker] = cnt
print speaker_dict
def map_speaker(speaker):
    if isinstance(speaker, basestring):
        speaker = speaker.lower()
        matches = [s for s in speakers if s in speaker]
        if len(matches) > 0:
            return speaker_dict[matches[0]] #Return index of first match
        else:
            return len(speakers)
    else:
        return len(speakers) #Nans or un-string data goes here.
data_set['speaker_id'] = data_set['speaker'].apply(map_speaker)
#val_set['speaker_id'] = val_set['speaker'].apply(map_speaker)
print len(speakers)

{'marco-rubio': 8, 'mitt-romney': 3, 'chain-email': 7, 'bernie-s': 11, 'newt-gingrich': 15, 'rick-perry': 6, 'hillary-clinton': 2, 'chris-christie': 12, 'facebook-posts': 13, 'charlie-crist': 14, 'john-mccain': 5, 'barack-obama': 0, 'joe-biden': 17, 'blog-posting': 18, 'paul-ryan': 19, 'jeb-bush': 16, 'donald-trump': 1, 'scott-walker': 4, 'rick-scott': 9, 'ted-cruz': 10}
20


In [0]:
data_set['speaker_job_title'].value_counts()[:10]
job_list = ['president', 'u.s. senator', 'governor', 'president-elect', 'presidential candidate', 
            'u.s. representative', 'state senator', 'attorney', 'state representative', 'congress']

job_dict = {'president':0, 'u.s. senator':1, 'governor':2, 'president-elect':3, 'presidential candidate':4, 
            'u.s. representative':5, 'state senator':6, 'attorney':7, 'state representative':8, 'congress':9}
#Possible groupings could be (11 groups)
#president, us senator, governor(contains governor), president-elect, presidential candidate, us representative,
#state senator, attorney, state representative, congress (contains congressman or congresswoman), rest
def map_job(job):
    if isinstance(job, basestring):
        job = job.lower()
        matches = [s for s in job_list if s in job]
        if len(matches) > 0:
            return job_dict[matches[0]] #Return index of first match
        else:
            return 10 #This maps any other job to index 10
    else:
        return 10 #Nans or un-string data goes here.
data_set['job_id'] = data_set['speaker_job_title'].apply(map_job)
#val_set['job_id'] = val_set['job'].apply(map_job)

In [0]:
data_set['part_affiliation'].value_counts()
#Possible groupings (6 groups)
#Hyper param -> num_party
party_dict = {'republican':0,'democrat':1,'none':2,'organization':3,'newsmaker':4}
#default index for rest party is 5
def map_party(party):
    if party in party_dict:
        return party_dict[party]
    else:
        return 5
data_set['party_id'] = data_set['part_affiliation'].apply(map_party)
#val_set['party_id'] = val_set['party'].apply(map_party)

In [0]:
#print data_set['state'].value_counts()[0:50]
#Possible groupings (50 groups + 1 for rest)
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho', 
         'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
         'Maine' 'Maryland','Massachusetts','Michigan','Minnesota',
         'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
         'New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio',    
         'Oklahoma','Oregon','Pennsylvania','Rhode Island',
         'South  Carolina','South Dakota','Tennessee','Texas','Utah',
         'Vermont','Virginia','Washington','West Virginia',
         'Wisconsin','Wyoming']
#states_dict = {}
#i = 0
#for state in states:
#    state_key = state.lower()
#    states_dict[state_key] = i
#    i += 1
#print len(states_dict.keys())

states_dict = {'wyoming': 48, 'colorado': 5, 'washington': 45, 'hawaii': 10, 'tennessee': 40, 'wisconsin': 47, 'nevada': 26, 'north dakota': 32, 'mississippi': 22, 'south dakota': 39, 'new jersey': 28, 'oklahoma': 34, 'delaware': 7, 'minnesota': 21, 'north carolina': 31, 'illinois': 12, 'new york': 30, 'arkansas': 3, 'west virginia': 46, 'indiana': 13, 'louisiana': 17, 'idaho': 11, 'south  carolina': 38, 'arizona': 2, 'iowa': 14, 'mainemaryland': 18, 'michigan': 20, 'kansas': 15, 'utah': 42, 'virginia': 44, 'oregon': 35, 'connecticut': 6, 'montana': 24, 'california': 4, 'massachusetts': 19, 'rhode island': 37, 'vermont': 43, 'georgia': 9, 'pennsylvania': 36, 'florida': 8, 'alaska': 1, 'kentucky': 16, 'nebraska': 25, 'new hampshire': 27, 'texas': 41, 'missouri': 23, 'ohio': 33, 'alabama': 0, 'new mexico': 29}
def map_state(state):
    if isinstance(state, basestring):
        state = state.lower()
        if state in states_dict:
            return states_dict[state]
        else:
            if 'washington' in state:
                return states_dict['washington']
            else:
                return 50 #This maps any other location to index 50
    else:
        return 50 #Nans or un-string data goes here.
data_set['state_id'] = data_set['state_info'].apply(map_state)
#val_set['state_id'] = val_set['state'].apply(map_state)

In [0]:
data_set['subject'].value_counts()[0:5]
#Possible groups (14)
subject_list = ['health','tax','immigration','election','education',
'candidates-biography','economy','gun','jobs','federal-budget','energy','abortion','foreign-policy']

subject_dict = {'health':0,'tax':1,'immigration':2,'election':3,'education':4,
'candidates-biography':5,'economy':6,'gun':7,'jobs':8,'federal-budget':9,'energy':10,'abortion':11,'foreign-policy':12}
#health-care,taxes,immigration,elections,education,candidates-biography,guns,
#economy&jobs ,federal-budget,energy,abortion,foreign-policy,state-budget, rest
#Economy & Jobs is bundled together, because it occurs together
def map_subject(subject):
    if isinstance(subject, basestring):
        subject = subject.lower()
        matches = [s for s in subject_list if s in subject]
        if len(matches) > 0:
            return subject_dict[matches[0]] #Return index of first match
        else:
            return 13 #This maps any other subject to index 13
    else:
        return 13 #Nans or un-string data goes here.

data_set['subject_id'] = data_set['subject'].apply(map_subject)
#val_set['subject_id'] = val_set['subject'].apply(map_subject)

In [0]:
data_set['context'].value_counts()[0:15]

venue_list = ['news release','interview','tv','radio',
              'campaign','news conference','press conference','press release',
              'tweet','facebook','email']
venue_dict = {'news release':0,'interview':1,'tv':2,'radio':3,
              'campaign':4,'news conference':5,'press conference':6,'press release':7,
              'tweet':8,'facebook':9,'email':10}
def map_venue(venue):
    if isinstance(venue, basestring):
        venue = venue.lower()
        matches = [s for s in venue_list if s in venue]
        if len(matches) > 0:
            return venue_dict[matches[0]] #Return index of first match
        else:
            return 11 #This maps any other venue to index 11
    else:
        return 11 #Nans or un-string data goes here.
#possibe groups (12)
#news release, interview, tv (television), radio, campaign, news conference, press conference, press release,
#tweet, facebook, email, rest
data_set['venue_id'] = data_set['context'].apply(map_venue)
#val_set['venue_id'] = val_set['venue'].apply(map_venue)

In [0]:
data_set.head(2)

,ID,label,statement,subject,speaker,speaker_job_title,state_info,part_affiliation,true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification,label_id,speaker_id,job_id,party_id,state_id,subject_id,venue_id
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30.1,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w...",5,6,2,0,41,2,1
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1.0,0,0,0,a news conference,She cited layoff notices received by the state...,1,20,8,1,47,8,5


In [0]:
data_set.drop(['ID','justification','statement','subject','speaker','speaker_job_title','state_info','part_affiliation'],axis=1,inplace= True)

In [0]:
data_set.drop(['context'],axis=1,inplace= True)

In [0]:
data_set.head(2)

,label,true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,label_id,speaker_id,job_id,party_id,state_id,subject_id,venue_id
0,true,30,30.1,42,23,18,5,6,2,0,41,2,1
1,false,2,1.0,0,0,0,1,20,8,1,47,8,5


In [0]:
data_set.to_csv('all_git_1.csv')